<a href="https://colab.research.google.com/github/ibitoladgr8/Machine-Learning/blob/main/Fake_News_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [11]:
df = pd.read_csv('/content/FakeNewsNet.csv')
df

,title,news_url,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38,1
...,...,...,...,...,...
23191,Pippa Middleton wedding: In case you missed it...,https://www.express.co.uk/news/royal/807049/pi...,www.express.co.uk,52,1
23192,Zayn Malik & Gigi Hadid’s Shocking Split: Why ...,hollywoodlife.com/2018/03/13/zayn-malik-gigi-h...,hollywoodlife.com,7,0
23193,Jessica Chastain Recalls the Moment Her Mother...,http://www.justjared.com/2018/01/17/jessica-ch...,www.justjared.com,26,1
23194,"Tristan Thompson Feels ""Dumped"" After Khloé Ka...",www.intouchweekly.com/posts/tristan-thompson-f...,www.intouchweekly.com,24,0


In [12]:
df.columns

Index(['title', 'news_url', 'source_domain', 'tweet_num', 'real'], dtype='object')

In [13]:
df = df.drop(['news_url', 'tweet_num'], axis =1)

In [18]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')
nltk.download('punkt')

port_stem=PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [16]:
import re

def stemming(content):
    con=re.sub('[^a-zA-Z]', ' ', content)
    con=con.lower()
    con=con.split()
    con=[port_stem.stem(word) for word in con if not word in stopwords.words('english')]
    con=' '.join(con)
    return con

In [19]:
df['title']= df['title'].apply(stemming)

In [21]:
import re

def extract_domain(url):
    # Check if url is a string or bytes-like object
    if isinstance(url, (str, bytes)):
        # Regex pattern to capture the domain name part
        pattern = r"(?:https?://)?(?:www\.)?([a-zA-Z0-9-]+)\."
        match = re.search(pattern, url)

        # If there's a match, return the domain name
        if match:
            return match.group(1)
        else:
            return None
    else:
        # Handle non-string values (e.g., return None or a default value)
        return None

# Apply the function to the 'source_domain' column to create a new 'source_domain' column
df['source_domain'] = df['source_domain'].apply(extract_domain)

In [23]:
# Assuming you want to join 'title' and 'source_domain' columns into a new column 'combined'
df['combined'] = df.apply(lambda row: ' '.join(str(x) for x in [row['title'], row['source_domain']]), axis=1)

In [25]:
df.drop(['title','source_domain'], axis=1, inplace=True)

In [26]:
from tkinter.constants import X
from sklearn.model_selection import train_test_split

X = df['combined']
y = df['real']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
vect = TfidfVectorizer()

X_train = vect.fit_transform(X_train)
X_test = vect.transform(X_test)



In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

model = RandomForestClassifier(class_weight='balanced', random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8568965517241379
